In [ ]:
import pandas as pd
import voyageai
from voyageai import get_embeddings
from transformers import AutoTokenizer, LlamaTokenizer, pipeline
import google.generativeai as genai
import transformers
import torch
import numpy as np
from numpy import pad
from openai import OpenAI
from huggingface_hub import login
import openai
from sentence_transformers import SentenceTransformer
import pickle, os

In [ ]:
device='cpu'

In [ ]:
%run ./config_embed.ipynb

In [ ]:
%run ./utils.ipynb

In [ ]:
dataset_name = 'cmu_book'
# dataset_name = 'cmu_movie'

In [ ]:
BENCHMARKING_TYPE = 'p_y_c_y_default'   # dataset_name = cmu book /cmu movie
# BENCHMARKING_TYPE = 'p_y_c_n_loc_exists_no' # cmu book only 
# BENCHMARKING_TYPE = 'p_y_c_n_country_exists_yes'  # cmu book only
# BENCHMARKING_TYPE = 'p_y_c_n_same_country_names'# cmu book only
# BENCHMARKING_TYPE = 'p_y_c_n_loc_any'

In [ ]:
if BENCHMARKING_TYPE == 'p_y_c_y_default':
    file_path = f'BENCHMARKING/{dataset_name}/{dataset_name}_dataset_with_p_y_c_y_perturbed_p_y_c_y.pkl'
    col_name_prefix = 'plot_perturb_p_True_c_True_id'


elif BENCHMARKING_TYPE == 'p_y_c_n_loc_exists_no':
    file_path = f'BENCHMARKING/{dataset_name}/{dataset_name}_dataset_with_p_y_c_n_perturbed_p_y.pkl'
    col_name_prefix = 'plot_perturb_p_True_c_False_id'


elif BENCHMARKING_TYPE == 'p_y_c_n_same_country_names':
    country_people =  'Spain'# 'India'#France'#, 'India', 'France']
    file_path = f'BENCHMARKING/{dataset_name}/{dataset_name}_dataset_with_p_y_c_n_using_names_from_{country_people}_perturbed_p_y.pkl'
    col_name_prefix = 'plot_perturb_p_True_c_False_id'
 
elif BENCHMARKING_TYPE == 'p_y_c_n_loc_any':

    file_path = f'BENCHMARKING/{dataset_name}/{dataset_name}_dataset_with_p_y_c_any_perturbed_p_y.pkl'
    col_name_prefix = 'plot_perturb_p_True_c_False_id'

else:
    raise Exception('Invalid benchmarking type')

In [ ]:

dataset = pd.read_pickle(file_path)
display(dataset)

In [ ]:
len(dataset)

In [ ]:

for model_name in EVAL_MODEL_LIST:
    if model_name not in dict_max_sequence_length:
        print('model_name ', model_name)
        if model_name in ['gemini']:
            token_limit = 2000
            dict_max_sequence_length[model_name] = token_limit

        elif model_name in ['voyage-3-lite']:
            token_limit =32000
            dict_max_sequence_length[model_name] = token_limit

        elif model_name in ['text-embedding-3-small', 'text-embedding-3-large']: # openai
            token_limit = 8191
            dict_max_sequence_length[model_name] = token_limit
            
        elif model_name in ['cohere']:
            token_limit =512
            dict_max_sequence_length[model_name] = token_limit
            
        else:
            model_object = SentenceTransformer(model_name, device='cpu')
            
            token_limit = model_object.max_seq_length
            del(model_object)
            print('token_limit ', token_limit)
            dict_max_sequence_length[model_name] = token_limit

        dict_max_sequence_length[model_name] = dict_max_sequence_length[model_name]*word_limit_multiplier

print('dict_max_sequence_length ', dict_max_sequence_length)

    

In [ ]:
sentence_transformer_models_dict = {}

In [ ]:
col_name_prefix

In [ ]:
dataset.columns

In [ ]:
col_name_prefix

In [ ]:
EVAL_MODEL_LIST

In [ ]:
BENCHMARKING_TYPE

In [ ]:
col_name_prefix

In [ ]:
for model_name in EVAL_MODEL_LIST:

    path_embed_prefix = f'BENCHMARKING/{dataset_name}/EMBEDDINGS_{BENCHMARKING_TYPE}/'

    if BENCHMARKING_TYPE =='p_y_c_n_same_country_names':
        path_embed_prefix = f'BENCHMARKING/{dataset_name}/EMBEDDINGS_{BENCHMARKING_TYPE}/{country_people}'
    
    os.makedirs(path_embed_prefix, exist_ok=True)

    output_file_name = f'{path_embed_prefix}/{dataset_name}_{model_name}'

    print('output_file_name ', output_file_name)

    dataset_with_embeddings = None

    print('model_name ', model_name, dict_max_sequence_length[model_name])
    dataset_with_embeddings = dataset.copy()
    dataset_with_embeddings = dataset_with_embeddings[dataset_with_embeddings.plot_length < dict_max_sequence_length[model_name] ]

    print('len applicable', len(dataset_with_embeddings))

    print('iterate ', model_name)
    for i in range(len(dataset_with_embeddings.columns)):
        col_name = f'{col_name_prefix}_{i}'
        print('col_name ', col_name)
        if col_name in dataset_with_embeddings.columns:
            embed_list = dataset_with_embeddings[col_name].tolist()
            
            embeddings = embed_content(embed_list, model_name)
            if model_name !='gemini' and model_name!='cohere' and model_name!='voyage-3-lite':
                embeddings = embeddings.tolist()

            dataset_with_embeddings[f'embed_{col_name}_model_{model_name}'] = embeddings

    try:
        with open(output_file_name, "wb") as f:
            pickle.dump(dataset_with_embeddings, f)
    except:
        print('error in pickling')            


In [ ]:
BENCHMARKING_TYPE